<center><img src="assets/header.png" width="25%" style="border-radius:20px;" /></center>

# Intro

We are Gal Elharar (207906553) and Roy Wolfer (324074566), computer science students with a strong passion for data science and its applications in healthcare.  
For our semester project, we sought to tackle a meaningful challenge while expanding our skill set.  
After exploring Kaggle competitions, we were immediately drawn to the [Equity in post-HCT Survival Predictions](https://www.kaggle.com/competitions/equity-post-HCT-survival-predictions) challenge.

This competition focuses on predicting survival probabilities for patients undergoing hematopoietic cell transplantation (HCT),  
with an emphasis on ensuring equitable outcomes across different racial groups.  
The evaluation metric used in this challenge is the Stratified Concordance Index (C-index),  
which adjusts for racial stratification to promote fairness and equity in the predictions.  
This makes the challenge particularly impactful in addressing disparities in healthcare outcomes.

We are excited to dive into this project, combining our data science skills with our interest in advancing equity in medicine.

### The problem

We couldn’t explain the problem any better as it was outlined in the kaggle challenge description:

Improving survival predictions for allogeneic HCT patients is a vital healthcare challenge.  
Current predictive models often fall short in addressing disparities related to socioeconomic status, race, and geography.  
Addressing these gaps is crucial for enhancing patient care, optimizing resource utilization, and rebuilding trust in the healthcare system.

This competition aims to encourage participants to advance predictive modeling by ensuring that survival predictions are both precise and fair for patients across diverse groups.  
By using synthetic data, which mirrors real-world situations while protecting patient privacy, participants can build and improve models that more effectively consider diverse backgrounds and conditions.

You’re challenged to develop advanced predictive models for allogeneic HCT that enhance both accuracy and fairness in survival predictions.  
The goal is to address disparities by bridging diverse data sources, refining algorithms, and reducing biases to ensure equitable outcomes for patients across diverse race groups.  
Your work will help create a more just and effective healthcare environment, ensuring every patient receives the care they deserve.

### Risk Score

In the “Equity in post-HCT Survival Predictions” challenge, survival predictions are quantified as risk scores.  
These scores represent the likelihood of an adverse event (i.e., death, relapse, rejection, or moderate to severe GVHD) occurring within a specific timeframe after hematopoietic cell transplantation (HCT).  
To evaluate the predictions, the risk scores are compared to actual survival outcomes, and their discrimination ability is assessed using the Stratified Concordance Index (C-index).

##### Key Details:

1. Risk Score Scale:
    - A higher risk score indicates a higher likelihood of the target event (e.g., shorter survival time or higher mortality risk).
    - A lower risk score suggests a lower probability of the target event (e.g., longer survival time or lower mortality risk).
2. Stratification for Fairness:
    - The risk scores are stratified across different racial groups to evaluate fairness.
    - Consistent and unbiased predictions across groups are critical for equitable healthcare applications.
3. Clinical Implications:
    - These risk scores may guide clinical decision-making by identifying patients at higher risk, enabling personalized follow-up and care.

# EDA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import numpy as np
import xgboost
import catboost
import warnings
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.utils import concordance_index
from scipy.stats import rankdata
from itables import show
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, quantile_transform
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import optuna
from xgboost import XGBRegressor
import statsmodels

all_model_scores = {}

### Reading the data

We read the data and observe:
1. The training dataset has 59 columns, many of which are categorical and have missing values.
2. Two columns are missing from the test dataset: `efs` and `efs_time`. These two columns together make up the target.

In [3]:
train = pd.read_csv('datasets/train.csv', index_col='ID')
test = pd.read_csv('datasets/test.csv', index_col='ID')
data_dictionary = pd.read_csv('datasets/data_dictionary.csv')
train.tail()

In [4]:
features = [f for f in test.columns if f != 'ID']

race_groups = np.unique(train.race_group)


### The data dictionary

The data dictionary simply describes the 59 columns of the dataset:

In [5]:
show(data_dictionary, scrollY="400px", scrollCollapse=True, paging=False, classes="display compact", columnDefs=[{"className": "dt-left", "targets": "_all"}], layout={"topStart": "search", "topEnd": None})

### Race group distribution

In the training data, there are six race groups with about 4800 samples each.  
Because in no country of the world these six race groups occur with equal frequencies, we know that some of the groups have been upsampled or downsampled in the dataset.

Note: The balanced representation of race groups in this dataset suggests that it has been engineered to ensure equal representation rather than reflecting real-world distributions. While this helps create fairer models, it also means that results might not generalize well to actual clinical populations. Any findings should be interpreted with caution, especially when assessing disparities across racial groups.

In [6]:
race_counts = train.race_group.value_counts()

plt.pie(race_counts.values, labels=race_counts.index, autopct='%1.1f%%', 
        startangle=140, colors=plt.cm.Paired.colors, wedgeprops={'edgecolor': 'black'})

plt.title('Race Group Distribution')
plt.show()

### Statistical Tests

Let's perform various statistical tests to validate our observations and understand relationships in the data:  
1. Chi-square tests for categorical variables to check if distributions differ across race groups  
2. ANOVA tests for numerical variables to check for significant differences across race groups  
3. Correlation analysis between numerical features  

In [ ]:
# Chi-square tests for categorical variables vs race_group
from scipy.stats import chi2_contingency

cat_vars = ['conditioning_intensity', 'graft_type', 'cmv_status', 'sex_match']
chi_results = {}

print("Chi-square tests for categorical variables vs race_group:")
print("-" * 50)
for var in cat_vars:
    # Create contingency table
    contingency = pd.crosstab(train['race_group'], train[var])
    # Perform chi-square test
    chi2, p_val, dof, expected = chi2_contingency(contingency)
    chi_results[var] = {'chi2': chi2, 'p_value': p_val}
    print("{var}:")
    print("Chi-square statistic: {chi2:.2f}")
    print("p-value: {p_val:.2e}")
    print("Significant: {'Yes' if p_val < 0.05 else 'No'}")
    print("-" * 50)

In [ ]:
# ANOVA tests for numerical variables across race groups
from sklearn.feature_selection import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

num_vars = ['age_at_hct', 'donor_age', 'comorbidity_score', 'karnofsky_score']

print("ANOVA tests for numerical variables across race groups:")
print("-" * 50)
for var in num_vars:
    # Collect groups
    groups = [group[var].dropna() for name, group in train.groupby('race_group')]
    # Perform ANOVA
    f_stat, p_val = f_oneway(*groups)
    print("{var}:")
    print("F-statistic: {f_stat:.2f}")
    print("p-value: {p_val:.2e}")
    print("Significant: {'Yes' if p_val < 0.05 else 'No'}")
    # If significant, perform Tukey's HSD test
    if p_val < 0.05:
        print("Tukey's HSD test results:")
        tukey = pairwise_tukeyhsd(endog=train[var].dropna(),
                                 groups=train['race_group'].dropna(),
                                 alpha=0.05)
        print(tukey)
    print("-" * 50)

In [ ]:
# Correlation analysis between numerical variables"
from sklearn.isotonic import spearmanr

numerical_vars = ['age_at_hct', 'donor_age', 'comorbidity_score', 'karnofsky_score']

print("Spearman correlation tests between numerical variables:")
print("-" * 50)
for i in range(len(numerical_vars)):
    for j in range(i+1, len(numerical_vars)):
        var1, var2 = numerical_vars[i], numerical_vars[j]
        correlation, p_val = spearmanr(train[var1].dropna(), train[var2].dropna())
        print("{var1} vs {var2}:")
        print("Correlation coefficient: {correlation:.3f}")
        print("p-value: {p_val:.2e}")
        print("Significant: {'Yes' if p_val < 0.05 else 'No'}")
        print("-" * 50)

1. **Categorical Variables (Chi-square tests)**:
   - These tests reveal if the distribution of categories (like conditioning intensity or graft type) differs significantly across race groups
   - A significant p-value (< 0.05) indicates that the distribution is not independent of race

2. **Numerical Variables (ANOVA tests)**:
   - These tests show if there are significant differences in means across race groups
   - For significant differences, Tukey's HSD test shows exactly which race groups differ
   - This is particularly important for variables like age and comorbidity scores

3. **Correlation Analysis**:
   - Spearman correlation tests reveal relationships between numerical variables
   - This helps understand which features might be redundant or complementary"

### Missing values

In [7]:
# Calculate missing values percentage
missing_values = train.isnull().sum() / len(train) * 100
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

# Generate a color gradient from dark blue (low missing) to red (high missing)
colors = sns.color_palette("coolwarm", len(missing_values))[::-1]

# Plot missing values with updated formatting
plt.figure(figsize=(14, 10))
bars = sns.barplot(y=missing_values.index, x=missing_values.values, palette=colors, hue=missing_values.index)

# Set x-axis to always go up to 100%
plt.xlim(0, 100)

# Labels and title
plt.xlabel("Percentage of Missing Values", fontsize=14)
plt.ylabel("Feature Name", fontsize=14)
plt.title("Missing Values Percentage per Feature", fontsize=16)

# Rotate x-axis ticks for better visibility
plt.xticks(fontsize=12)
plt.yticks(fontsize=12, rotation=0)

# Annotate each bar with its missing percentage
for bar, value in zip(bars.patches, missing_values.values):
    plt.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
             f"{value:.1f}%", ha='left', va='center', fontsize=12, color='black')

plt.show()

Some features are incomplete. For example, we see that `tce_match` has a lot of missing values!  
*(TCE Match refers to a specific type of immune match between donors and recipients in HCT transplants)*  

What should we do?

We have three options:
1. Fill in (impute) the missing values using the mean, median, or mode.
2. Drop the feature if it’s mostly empty.
3. Treat missing values as a separate category - sometimes missing data itself carries meaning!

For now, at least for the categorical features, we choose the third option.

In [8]:
cat_features = list(train.select_dtypes(object).columns)
train[cat_features] = train[cat_features].astype(str).astype('category')

Let's review the missing values now.

In [9]:
# Calculate missing values percentage
missing_values = train.isnull().sum() / len(train) * 100
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

# Generate a color gradient from dark blue (low missing) to red (high missing)
colors = sns.color_palette("coolwarm", len(missing_values))[::-1]

# Plot missing values with updated formatting
plt.figure(figsize=(14, 10))
bars = sns.barplot(y=missing_values.index, x=missing_values.values, palette=colors, hue=missing_values.index)

# Set x-axis to always go up to 100%
plt.xlim(0, 100)

# Labels and title
plt.xlabel("Percentage of Missing Values", fontsize=14)
plt.ylabel("Feature Name", fontsize=14)
plt.title("Missing Values Percentage per Feature", fontsize=16)

# Rotate x-axis ticks for better visibility
plt.xticks(fontsize=12)
plt.yticks(fontsize=12, rotation=0)

# Annotate each bar with its missing percentage
for bar, value in zip(bars.patches, missing_values.values):
    plt.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
             f"{value:.1f}%", ha='left', va='center', fontsize=12, color='black')

plt.show()

Much better!  
Now our models will be able to handle missing values for categorical features.

### The weirdness of the age distribution

There are only two features with continuous data: donor age and patient age.  
The patient age histogram shows that the patient age distribution has five modes.  
Such a distribution is highly unnatural — it must be an artefact of the synthetic data generation. 

Since this dataset was synthetically generated, the age distribution does not fully reflect real-world patient demographics. This is crucial to consider when interpreting model results, as certain patterns may be artifacts of data preprocessing rather than genuine clinical insights.

In [10]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.hist(train.donor_age, bins=50, color='skyblue')
plt.title('Donor age histogram')
plt.xlabel('donor_age')
plt.ylabel('count')
plt.subplot(1, 2, 2)
plt.title('Patient age histogram')
plt.hist(train.age_at_hct, bins=50, color='skyblue')
plt.xlabel('age_at_hct')
plt.tight_layout()
plt.show()

Our first thought was that different race groups had different modes, but the patient age distribution has the same five modes in every race group:

In [11]:
_, axs = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(12, 9))
for race_group, ax in zip(race_groups, axs.ravel()):
    ax.hist(train.age_at_hct[train.race_group == race_group],
            bins=np.linspace(0, 74, 38),
            color='skyblue')
    ax.set_title(f'Patient age histogram for {race_group}')
    ax.set_xlabel('age_at_hct')
    ax.set_ylabel('count')
plt.tight_layout()
plt.show()

Even stranger: The age of 0.044 years (i.e., 16 days) occurs 1005 times in the training dataset, whereas every other age occurs at most six times.  
Is hematopoietic cell transplantation a treatment which is often done to newborns? Possible. But we can't believe that these babies are all treated exactly when they are 16 days old. 

In [12]:
train.age_at_hct.value_counts().sort_values(ascending=False).head()

### The prediction target

The prediction target consists of two parts:
1. `efs_time`, always positive, is a time, measured in months.
2. `efs`, always zero or one, indicates the presence or absence of an event:
   - `efs=1` means "patient was without event for **exactly** `efs_time`.
   - `efs=0` means "patient was without event for **at least** `efs_time`"

This situation is called "censored data": Samples of which we know the time of event are uncensored, and if we only know a lower bound for the time of event, the sample is (right-)censored.

**Censoring is the main reason that this competition has a special metric and that we need special models.**  
The competition is a regression task, but we know y_true for only half the samples.  
For the other (censored) half, all we know is lower bounds for y_true.  
One cannot compute a squared error based on `y_true > 100 and y_pred == 120`. RMSE and similar metrics cannot deal with that.

Weird thing to note, `efs_time` is a float with three digits after the decimal point.  
Even though it is measured in months, the recorded numbers don't really align to days for any month.  
We doubt that possible events of a patient are recorded with a more exact unit than a day.  
So this might also be a case caused by the synthetic data generation.

A histogram of the target values shows that half the patients have an event within 20 months after the transplantation.  
The other half, without event for the first 20 months, has a high probability of not having an event for much longer.

In [13]:
plt.figure(figsize=(6, 3))
plt.hist(train.efs_time[train.efs == 0], bins=np.linspace(0, 160, 41), label='efs=0: patient has no event for at least this time', alpha=0.5)
plt.hist(train.efs_time[train.efs == 1], bins=np.linspace(0, 160, 41), label='efs=1: patient has event at this time', alpha=0.5)
plt.legend()
plt.xlabel('efs_time')
plt.ylabel('count')
plt.title('Target histogram')
plt.show()

### Survival function and cumulative hazard function

The survival function shows how many patients have no event for an amount of time ([Kaplan–Meier estimator](https://en.wikipedia.org/wiki/Kaplan%E2%80%93Meier_estimator)).  
At month 0, 100% of the patients have no event. At month 20, only 40% - 60% remain without event, depending on their race group.  
Patients with "more than one race" have the highest probability of event-free survival, while whites have the lowest.

The cumulative hazard is another representation of the same facts, it corresponds to the negative logarithm of the survival function ([Nelson–Aalen estimator](https://en.wikipedia.org/wiki/Nelson%E2%80%93Aalen_estimator)).

In [14]:
def survival_function(df):
    survival_df = df[['efs', 'efs_time']].groupby('efs_time').agg(['size', 'sum']).droplevel(0, axis=1).astype(int)
    survival_df['n_at_risk'] = survival_df['size'].sum() - survival_df['size'].shift().fillna(0).cumsum().astype(int)
    hazard = survival_df['sum'] / survival_df['n_at_risk'] 
    survival_df['cumulative_hazard'] = np.cumsum(hazard) # nelson_aalen_estimator
    survival_df['survival_probability'] = (1 - hazard).cumprod() # kaplan_meier_estimator
    return survival_df

plt.figure(figsize=(6, 8))

plt.subplot(2, 1, 1)
survival_df = survival_function(train)
plt.step(survival_df.index, survival_df['survival_probability'], c='k', where="post", label='[Overall]')
plt.xlabel('efs_time')
for race_group in race_groups:
    subset = train.query('race_group == @race_group')
    survival_df = survival_function(subset)
    plt.step(survival_df.index, survival_df['survival_probability'], where="post", label=race_group)
plt.xlabel('efs_time')
plt.legend(loc='upper right')
plt.title('Survival function (Kaplan–Meier) by race group')
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0)) # percent of xmax

plt.subplot(2, 1, 2)
survival_df = survival_function(train)
plt.step(survival_df.index, survival_df['cumulative_hazard'], c='k', where="post", label='[Overall]')
plt.xlabel('efs_time')
for race_group in race_groups:
    subset = train.query('race_group == @race_group')
    survival_df = survival_function(subset)
    plt.step(survival_df.index, survival_df['cumulative_hazard'], where="post", label=race_group)
plt.xlabel('efs_time')
plt.legend(loc='lower right')
plt.title('Cumulative hazard (Nelson–Aalen) by race group')

plt.tight_layout()
plt.show()


### Is the Difference in Survival Significant?  
So far, we've seen that survival probabilities **vary by race group**, but are these differences **statistically significant**?  
To answer this, we'll use the **Log-Rank Test**, a statistical test designed to compare survival distributions between groups.  

If the p-value is **less than 0.05**, it means the difference is **statistically significant**—one group truly has a better survival outcome than the other. If not, any differences we see could be due to chance.

In [15]:
from lifelines.statistics import logrank_test

# Select two race groups for comparison
race_group_1 = train[train["race_group"] == "White"]
race_group_2 = train[train["race_group"] == "More than one race"]

# Perform Log-Rank Test
log_rank_result = logrank_test(
    race_group_1["efs_time"], race_group_2["efs_time"],
    event_observed_A=race_group_1["efs"],
    event_observed_B=race_group_2["efs"]
)

# Print p-value to check statistical significance
print(f"Log-Rank Test p-value: {log_rank_result.p_value}")

# Interpretation of results
if log_rank_result.p_value < 0.05:
    print("✅ Significant survival difference! One race group has a statistically different survival rate.")
else:
    print("❌ No significant difference in survival between these race groups.")

Findings:
- The p-value obtained from the test is 1.79e-49, which is extremely small (<< 0.05).
-	This means that the difference in survival rates between the "White" group and the "More than one race" group is statistically significant.
-	In other words, one of these groups experiences better survival outcomes than the other, and this difference is unlikely due to random chance.

### Feature correlation
Sometimes, two features are so similar that they provide the same information.  
If this happens, we might remove one to make our model simpler.

Let's explore the relationships between numerical features.  
Correlations can help us identify redundant features, where one variable provides almost the same information as another.  
High correlation (close to 1 or -1) means we might consider removing one of the features to avoid redundancy.

In [16]:
# Generate a correlation heatmap
plt.figure(figsize=(12, 8))
corr_matrix = train.corr(numeric_only=True) # Only numerical columns
sns.heatmap(corr_matrix, cmap="coolwarm", annot=False, linewidths=0.5)
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

1. 	Some features have a very high correlation:  
	Many HLA match features (`hla_match_*`, `hla_high_res_*`) are strongly correlated.  
	This makes sense! They’re all measuring similar compatibility factors between donors and patients.  
	Do we really need all of them? Maybe not! We might remove some duplicates to keep things simple.

2. 	Survival time (`efs_time`) seems Hard to predict:  
	`efs_time` doesn’t have strong correlations with most features.  
	This tells us that survival time depends on many small factors rather than one big predictor.

3.  Age and donor age:  
	Patients (`age_at_hct`) and their donors (`donor_age`) show some relationship, but it's not very strong.

4. 	Health scores:  
	Comorbidity score and Karnofsky score have some level of correlation with each other.  
	This makes sense! A patient with more health issues (high comorbidity) probably has a lower Karnofsky score (worse physical condition).

Important Consideration Before Model Evaluation:

Since the dataset is synthetic, some of the patterns our models learn may not generalize to real-world patient outcomes. This is especially important when interpreting survival probabilities, as factors influencing patient outcomes in this dataset may differ from those in actual clinical settings.

# Evaluation
This competition is about equity in the predictions. This means that we score the predictions per race group and then derive the final score from these six sub-scores.  
As the official implementation of the competition metric doesn't output the scores per race group, we've written our own implementation, which gives more transparency.

We also show the scores recieved for each fold in the cross validation.
The overall score is the mean of all of the fold's scores.

In [17]:
kf = StratifiedKFold(n_splits=5 ,shuffle=True, random_state=1)
all_scores = {}
X_va = {}
idx_va = {}

def evaluate_fold(y_va_pred, fold):
    """Compute and print the metrics (concordance index) per race group for a single fold.

    Global variables:
    - train, X_va, idx_va
    - The metrics are saved in the global list all_scores.
    """
    metric_list = []
    for race in race_groups:
        mask = X_va.race_group.values == race
        c_index_race = concordance_index(
            train.efs_time.iloc[idx_va][mask],
            - y_va_pred[mask],
            train.efs.iloc[idx_va][mask]
        )
        # print(f"# {race:42} {c_index_race:.3f}")
        metric_list.append(c_index_race)
    fold_score = np.mean(metric_list) - np.sqrt(np.var(metric_list))
    print(f"# Total fold {fold}:{' ':29} {fold_score:.3f} mean={np.mean(metric_list):.3f} std={np.std(metric_list):.3f}")
    all_scores.append(metric_list)

def display_overall(label):
    """Compute and print the overall metrics (concordance index)"""
    df = pd.DataFrame(all_scores, columns=race_groups)
    df['mean'] = df[race_groups].mean(axis=1)
    df['std'] = np.std(df[race_groups], axis=1)
    df['score'] = df['mean'] - df['std']
    df = df.T
    df['Overall'] = df.mean(axis=1)
    temp = df.drop(index=['std']).values
    print(f"# Overall:                                   {df.loc['score', 'Overall']:.3f} {label}")
    all_model_scores[label] = df.loc['score', 'Overall']
    display(df
            .iloc[:len(race_groups)]
            .style
            .format(precision=3)
            .background_gradient(axis=None, vmin=temp.min(), vmax=temp.max(), cmap="cool")
            .concat(df.iloc[len(race_groups):].style.format(precision=3))
           )
    

# Cox proportional hazards model
One method for survival analysis, is the [Cox proportional hazards model](https://en.wikipedia.org/wiki/Proportional_hazards_model).  
Which is implemented in XGBoost and in CatBoost.

Let's try the first method.

### XGBoost Cox proportional hazards model
Cox proportional hazards model with XGBoost.  
This model expects that the two target columns be combined into one (`y = np.where(train.efs == 1, train.efs_time, -train.efs_time)`, negative target values are considered right censored)

In [ ]:
# %%time
y = np.where(train.efs == 1, train.efs_time, -train.efs_time)
all_scores = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = y[idx_tr]
    
    xgb_cox_params = {'objective': 'survival:cox', 'grow_policy': 'depthwise', 
                      'n_estimators': 700, 'learning_rate': 0.0254, 'max_depth': 8, 
                      'reg_lambda': 0.116, 'reg_alpha': 0.139, 'min_child_weight': 23.8,
                      'colsample_bytree': 0.59, 'subsample': 0.7, 'tree_method': 'hist',
                      'enable_categorical': True}
    model = xgboost.XGBRegressor(**xgb_cox_params)
    model.fit(X_tr, y_tr) # negative values are considered right censored
    y_va_pred = model.predict(X_va) # predicts hazard factor
    evaluate_fold(y_va_pred, fold)
display_overall('Cox Proportional Hazards XGBoost')

In [20]:
# %%time
y = np.where(train.efs == 1, train.efs_time, -train.efs_time)
all_scores = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = y[idx_tr]
    
    xgb_cox_params = {'objective': 'survival:cox', 'grow_policy': 'depthwise', 
                      'n_estimators': 700, 'learning_rate': 0.0254, 'max_depth': 8, 
                      'reg_lambda': 0.116, 'reg_alpha': 0.139, 'min_child_weight': 23.8,
                      'colsample_bytree': 0.59, 'subsample': 0.7, 'tree_method': 'hist',
                      'enable_categorical': True}
    model = xgboost.XGBRegressor(**xgb_cox_params)
    model.fit(X_tr, y_tr) # negative values are considered right censored
    y_va_pred = model.predict(X_va) # predicts hazard factor
    evaluate_fold(y_va_pred, fold)
display_overall('Cox Proportional Hazards XGBoost')

### Catboost Cox proportional hazards model
Cox proportional hazards model with CatBoost.  
This model expects the targets in the same format as the XGBoost Cox model.

In [21]:
# %%time
y = np.where(train.efs == 1, train.efs_time, -train.efs_time)
all_scores = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = y[idx_tr]
    
    cb_cox_params = {'loss_function': 'Cox', 'grow_policy': 'SymmetricTree',
                     'n_estimators': 800, 'learning_rate': 0.092, 'l2_leaf_reg': 2.5,
                     'max_depth': 7, 'colsample_bylevel': 0.84, 'subsample': 0.9, 
                     'random_strength': 0.8, 'verbose': False}
    
    model = catboost.CatBoostRegressor(**cb_cox_params, cat_features=cat_features)
    model.fit(X_tr, y_tr)
    y_va_pred = model.predict(X_va) # predicts log of hazard factor
    evaluate_fold(y_va_pred, fold)
display_overall('Cox Proportional Hazards CatBoost')

# Accelerated failure time model
Another method for survival analysis, is the [Accelerated failure time model](https://en.wikipedia.org/wiki/Accelerated_failure_time_model).  
Which is also implemented in XGBoost and in CatBoost.

We shall try it too.

### XGBoost Accelerated failure time model
Accelerated failure time model with XGBoost.  
This model expects the lower and upper bounds for the target in a special form in a DMatrix.

In [22]:
# %%time
all_scores = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    d_tr = xgboost.DMatrix(X_tr, enable_categorical=True)
    d_tr.set_float_info('label_lower_bound', train.efs_time.iloc[idx_tr])
    d_tr.set_float_info('label_upper_bound', np.where(train.efs.iloc[idx_tr] == 0, np.inf, train.efs_time.iloc[idx_tr]))
    d_va = xgboost.DMatrix(X_va, enable_categorical=True)
    d_va.set_float_info('label_lower_bound', train.efs_time.iloc[idx_va])
    d_va.set_float_info('label_upper_bound', np.where(train.efs.iloc[idx_va] == 0, np.inf, train.efs_time.iloc[idx_va]))
    xgboost_aft_params = {'learning_rate': 0.08, 'max_depth': 4, 'reg_lambda': 3, 'aft_loss_distribution_scale': 0.9,
                          'reg_alpha': 0.24, 'gamma': 0.033, 'min_child_weight': 82.58861553592878,
                          'colsample_bytree': 0.5662198438953138, 'max_bin': 53, 'subsample': 0.7456329821182728, 
                          'objective': 'survival:aft', 'grow_policy': 'depthwise', 'tree_method': 'hist',
                          'aft_loss_distribution': 'normal'}
    bst = xgboost.train(xgboost_aft_params,
                        d_tr,
                        num_boost_round=300,
                       )
    y_va_pred = - bst.predict(d_va) # model predicts time of event
    evaluate_fold(y_va_pred, fold)
display_overall('Accelerated Failure Time XGBoost')

### CatBoost Accelerated failure time model
Accelerated failure time model with CatBoost.  
This model expects the lower and upper bounds for the target in the form of a two-column array.

In [23]:
# %%time
y = np.column_stack([train.efs_time,
                     np.where(train.efs == 1, train.efs_time, -1)])
all_scores = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = y[idx_tr]
    cb_aft_params = {'loss_function': 'SurvivalAft', 'grow_policy': 'SymmetricTree', 
                     'n_estimators': 800, 'learning_rate': 0.066, 'l2_leaf_reg': 4.4,
                     'max_depth': 5, 'colsample_bylevel': 0.776, 'random_strength': 0.9, 
                     'verbose': False}
    model = catboost.CatBoostRegressor(**cb_aft_params, cat_features=cat_features)
    model.fit(X_tr, y_tr)
    y_va_pred = - model.predict(X_va) # model predicts log of time of event
    evaluate_fold(y_va_pred, fold)
display_overall('Accelerated Failure Time CatBoost')

# Target transformation models and regression with mean squared error

The competition task can be interpreted as predicting the order of events for the patients.  
Who has an event first? Who second? ... Who has an event last? and who survives without an event at all?  
With a suitable target transformation, we can apply the usual regression algorithms which optimize MSE or similar metrics.

### Target transformations
As we saw in the EDA, patients with a known event, mostly have an `efs_time` between 0 and 15, whereas most unkowns have an `efs_time` between 15 and 160.  
This distribution is problematic for regression models.  
We want predictions to have high discriminative power for the patients who have a known event, but we don't need to distinguish between unkowns.  
We can achieve this result by stretching the range of the patients with a known event and compressing the range of the unkowns.

Let's create such a transformation to use for our model:  

In [24]:
def transform_survival_probability(time, event):
    """Transform the target by stretching the range of eventful efs_times and compressing the range of event_free efs_times"""
    kmf = KaplanMeierFitter()
    kmf.fit(time, event)
    return kmf.survival_function_at_times(time).values

The next diagram shows how a typical target transformation stretches and compresses the ranges.  
We use the transformation above as the example:

In [25]:
y_quantile = transform_survival_probability(time=train.efs_time, event=train.efs)
survival_df = survival_function(train)

fig, axs = plt.subplots(2, 2, figsize=(10, 10), dpi=80)

axs[0, 0].hist(train.efs_time[train.efs == 0], bins=np.linspace(0, 160, 41), label='efs=0: patient has no event for this time', alpha=0.5)
axs[0, 0].hist(train.efs_time[train.efs == 1], bins=np.linspace(0, 160, 41), label='efs=1: patient has event at this time', alpha=0.5)
axs[0, 0].legend()
axs[0, 0].set_xlabel('efs_time')
axs[0, 0].set_ylabel('count')
axs[0, 0].set_title('Original target histogram')

axs[0, 1].set_axis_off()

axs[1, 0].step(survival_df.index, survival_df['survival_probability'], c='k', lw=3, where="post", label='[Overall]')
axs[1, 0].set_xlabel('efs_time')
axs[1, 0].set_ylabel("quantile")
axs[1, 0].set_title("Survival function")
axs[1, 0].yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))

axs[1, 1].hist(y_quantile[train.efs==0], bins=100, label="efs=0", orientation=u'horizontal', alpha=0.5)
axs[1, 1].hist(y_quantile[train.efs==1], bins=100, label="efs=1", orientation=u'horizontal', alpha=0.5)
axs[1, 1].legend()
axs[1, 1].set_ylabel("quantile")
axs[1, 1].set_xlabel("count")
axs[1, 1].set_title("Transformed target histogram (sideways)")
axs[1, 1].yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))

ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
fig.add_axes(ax)
ax.arrow(0.2, 0.55, 0, -0.47, length_includes_head=True, width=0.002, color=plt.get_cmap('tab10')(0), alpha=0.5, head_width=0.02, head_length=0.02)
ax.arrow(0.2, 0.082, 0.37, 0, length_includes_head=True, width=0.002, color=plt.get_cmap('tab10')(0), alpha=0.5, head_width=0.02, head_length=0.02)
ax.arrow(0.12, 0.55, 0, -0.3, length_includes_head=True, width=0.002, color=plt.get_cmap('tab10')(1), alpha=0.5, head_width=0.02, head_length=0.02)
ax.arrow(0.12, 0.25, 0.45, 0, length_includes_head=True, width=0.002, color=plt.get_cmap('tab10')(1), alpha=0.5, head_width=0.02, head_length=0.02)

plt.suptitle('Transforming the target', y=0.99, size=20)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    plt.tight_layout()
plt.show()

We will add 4 more possible target transformations to test:

In [26]:
def transform_partial_hazard(time, event):
    """Transform the target by stretching the range of eventful efs_times and compressing the range of event_free efs_times"""
    data = pd.DataFrame({'efs_time': time, 'efs': event, 'time': time, 'event': event})
    cph = CoxPHFitter()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        cph.fit(data, duration_col='time', event_col='event')
    return cph.predict_partial_hazard(data)

def transform_separate(time, event):
    """Transform the target by separating events from non-events"""
    transformed = time.values.copy()
    mx = transformed[event == 1].max() # last patient that has a known event
    mn = transformed[event == 0].min() # first patient that does not have a known event
    transformed[event == 0] = time[event == 0] + mx - mn
    transformed = rankdata(transformed)
    transformed[event == 0] += len(transformed) // 2
    transformed = transformed / transformed.max()
    return - transformed

def transform_rank_log(time, event):
    """Transform the target by stretching the range of eventful efs_times and compressing the range of event_free efs_times"""
    transformed = time.values.copy()
    mx = transformed[event == 1].max() # last patient that has a known event
    mn = transformed[event == 0].min() # first patient that does not have a known event
    transformed[event == 0] = time[event == 0] + mx - mn
    transformed = rankdata(transformed)
    transformed[event == 0] += len(transformed) * 2
    transformed = transformed / transformed.max()
    transformed = np.log(transformed)
    return - transformed

def transform_quantile(time, event):
    """Transform the target by stretching the range of eventful efs_times and compressing the range of event_free efs_times"""
    transformed = np.full(len(time), np.nan)
    transformed_with_event = quantile_transform(- time[event == 1].values.reshape(-1, 1)).ravel()
    transformed[event == 1] = transformed_with_event
    transformed[event == 0] = transformed_with_event.min() - 0.3
    return transformed


### XGBoost MSE loss with five different target transformations
We now plot the histograms of the five possible transformations we have created, and then fit regression models with MSE loss to each of the transformed targets.

In [27]:
for transformation in [transform_survival_probability,
                       transform_partial_hazard,
                       transform_separate,
                       transform_rank_log,
                       transform_quantile,
                      ]:
    plt.figure(figsize=(6, 1.5))
    target = transformation(time=train.efs_time, event=train.efs)
    vmin, vmax = 1.09 * target.min() - 0.09 * target.max(), 1.09 * target.max() - 0.09 * target.min()
    plt.hist(target[train.efs == 0], bins=np.linspace(vmin, vmax, 31), density=True, label='efs=0: patient has no event for this time', alpha=0.5)
    plt.hist(target[train.efs == 1], bins=np.linspace(vmin, vmax, 31), density=True, label='efs=1: patient has event at this time', alpha=0.5)
    plt.xlim(vmin, vmax)
    plt.yticks([])
    plt.title('Target histogram: ' + transformation.__name__)
    plt.show()
    
    print(transformation.__name__)

    all_scores = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
        X_tr = train.iloc[idx_tr][features]
        X_va = train.iloc[idx_va][features]
        y_tr = transformation(time=train.iloc[idx_tr].efs_time, event=train.iloc[idx_tr].efs)
    
        model = xgboost.XGBRegressor(
            max_depth=3,  
            colsample_bytree=0.5,  
            subsample=0.8,  
            n_estimators=2000,  
            learning_rate=0.02,  
            enable_categorical=True,
            min_child_weight=80,
        )
        model.fit(X_tr, y_tr)
        y_va_pred = model.predict(X_va) # predicts quantile
        evaluate_fold(y_va_pred, fold)
    display_overall(f'{transformation.__name__} XGBoost (MSE)')
    print()

# A linear model

The linear model [CoxPHFitter](https://lifelines.readthedocs.io/en/latest/Survival%20Regression.html#cox-s-proportional-hazard-model) needs one-hot encoding and missing value imputation.  
This is a Cox proportional hazards model with a linear implementation. This model expects time and event columns in a dataframe.

In [28]:
# %%time
all_scores = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
    preproc = ColumnTransformer([('ohe', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_features),
                                ],
                                remainder=SimpleImputer(strategy='median'),
                                verbose_feature_names_out=False
                               ).set_output(transform='pandas')
    X_tr = preproc.fit_transform(train.iloc[idx_tr])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        X_va = preproc.transform(train.iloc[idx_va])
    model = CoxPHFitter(penalizer=.01)
    feats = [f for f in X_tr.columns if f not in ['gvhd_proph_FK+- others(not MMF,MTX)']]
    model.fit(X_tr[feats], duration_col='efs_time', event_col='efs')
    # model.print_summary()
    y_va_pred = model.predict_partial_hazard(X_va[feats])
    X_va['race_group'] = train.race_group.iloc[idx_va]
    evaluate_fold(y_va_pred, fold)
display_overall('Cox Proportional Hazards Linear')


# Conclusions

### Final comparison

For the time being, the gradient-boosted proportional hazard models (Cox regression, blue) and the transformed-target models (pink) win.  
Among the target transformations, `transform_quantile` is best.  
The AFT models (green) perhaps need more hyperparameter tuning.

In [29]:
result_df = pd.DataFrame(all_model_scores, index=['score']).T
result_df = result_df.sort_values('score', ascending=False)
# with pd.option_context("display.precision", 3): display(result_df)
plt.figure(figsize=(6, len(result_df) * 0.4))

color = np.where(result_df.index.str.contains('Proportional'),
                 'cyan',
                 np.where(result_df.index.str.contains('Accelerated'), 'lightgreen', 
                          'lightpink'))
bars = plt.barh(np.arange(len(result_df)), result_df.score, color=color)
plt.gca().bar_label(bars, fmt='%.3f')
plt.yticks(np.arange(len(result_df)), result_df.index)
plt.xlim(0.65, 0.68)
plt.xticks([0.65, 0.66, 0.67, 0.68])
plt.gca().invert_yaxis()
plt.xlabel('CV score (higher is better)')
plt.show()

### Feature importance
As `transform_quantile` was our best model,  
let's delve deeper into what features it cared about most:

In [30]:
feature_importances = []

for fold, (idx_tr, idx_va) in enumerate(kf.split(train, train.race_group)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = transform_quantile(time=train.iloc[idx_tr].efs_time, event=train.iloc[idx_tr].efs)

    model = xgboost.XGBRegressor(
        max_depth=3,  
        colsample_bytree=0.5,  
        subsample=0.8,  
        n_estimators=2000,  
        learning_rate=0.02,  
        enable_categorical=True,
        min_child_weight=80,
    )
    model.fit(X_tr, y_tr)
    
    # Store feature importance
    feature_importances.append(model.feature_importances_)

# Convert to numpy array and compute mean importance
feature_importances = np.array(feature_importances)
mean_importance = feature_importances.mean(axis=0)

# Create DataFrame for mean feature importance
importance_df = pd.DataFrame({
    "Feature": features,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# Normalize importance values for colormap
norm = plt.Normalize(importance_df["Importance"].min(), importance_df["Importance"].max())
colors = plt.cm.viridis(norm(importance_df["Importance"]))

plt.figure(figsize=(18, 15))
bars = plt.barh(importance_df["Feature"], importance_df["Importance"], color=colors)

plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("transform_quantile XGBoost (MSE) Feature Importance")
plt.gca().invert_yaxis()  # Flip features for better readability

# Add labels to each bar
for bar in bars:
    plt.text(
        bar.get_width() + 0.001,  # Position the text slightly to the right of the bar
        bar.get_y() + bar.get_height() / 2,  # Center the text vertically
        f"{bar.get_width():.4f}",  # Format the value
        va="center",  # Align vertically
        ha="left",  # Align horizontally
        fontsize=10
    )

plt.show()

These results make a lot of sense!  
The `conditioning_intensity` describes how a patient's body was prepared for HCT.  
Other top features include various scores (`dri_score`, `comorbidity_score`, etc.) that are given to patients to determine their health before HCT.  
It fits that they contribute the most to the score as they reflect the physical state the patient was in before HCT.

### Race group inequality
With most models, the Asian predictions get the highest scores (best concordance index) and the predictions for white patients get the lowest scores (worst concordance). 

As the competition's objective (equitability across diverse patient populations) rewards models with similar concordance scores for all six race groups,  
a possible strategy could be to artificially make the predictions for Asian patients worse.  

### Working environment
As we heard many disrecommendations for Kaggle notebooks as a work environment, we immediately skipped that option.

First we tried Google collab, but it proved difficult to work in,  
as the platform has bad intellisense, and no real realtime abillities.

We decided to switch to a rather newer option, Deepnote.  
It was easier to work there, due to having built in intellisense, realtime collaboration, and pretty UI.  
But for some reason, they made it really hard to work with text/markdown blocks.

We ended up working locally in VSCode by setting up a virtual environment,  
and using the Live Share extension for realtime collaboration.  
It wasn't super hard to set up, and is pretty easy to work with.  
So in the future, we will start with this option.